In [21]:
import pandas as pd

In [22]:
races = ['BL', 'WH', 'AM', 'AS', 'HI', 'HP', 'TR']
genders = ['M', 'F', 'both']
school_metrics = ['MULTOOS', 'SINGOOS', 'REF', 'GTENR', 'APENR']
disability = ['WODIS', 'WDIS']

In [23]:
query_segments = []

In [25]:
for metric in school_metrics:
    for race in races:
        for dis_status in disability:
            for gender in genders:
                if gender == 'both':
                    district_query = f',("SCH_DISC{dis_status}_{metric}_{race}_F" + "SCH_DISC{dis_status}_{metric}_{race}_M") / (NULLIF("SCH_ENR_{race}_F", 0) + NULLIF("SCH_ENR_{race}_M", 0))'

                else:
                    district_query = f',("SCH_DISC{dis_status}_{metric}_{race}_{gender}" / (NULLIF("SCH_ENR_{race}_{gender}", 0)'

                query_segments.append(district_query)
            break

In [27]:
len(query_segments)

210

In [ ]:
select "LEAID"
    ,"SCH_ENR_BL_F" + "SCH_ENR_BL_M" as bl_enrollment
    ,"SCH_ENR_WH_F" + "SCH_ENR_WH_M" as wh_enrollment
    ,("SCH_DISCWODIS_MULTOOS_BL_F" + "SCH_DISCWODIS_MULTOOS_BL_M") / (NULLIF("SCH_ENR_BL_F", 0) + NULLIF("SCH_ENR_BL_M", 0)) as bl_no_disab_mult_suspension_rate
    ,("SCH_DISCWODIS_MULTOOS_WH_F" + "SCH_DISCWODIS_MULTOOS_WH_M") / (NULLIF("SCH_ENR_Wh_F", 0) + NULLIF("SCH_ENR_WH_M", 0)) as wh_no_disab_mult_suspension_rate
    ,("SCH_DISCWODIS_SINGOOS_WH_F" + "SCH_DISCWODIS_SINGOOS_WH_M") / (NULLIF("SCH_ENR_WH_F", 0) + NULLIF("SCH_ENR_WH_M", 0)) as wh_no_disab_sing_suspension_rate
    ,("SCH_DISCWODIS_SINGOOS_BL_F" + "SCH_DISCWODIS_SINGOOS_BL_M") / (NULLIF("SCH_ENR_BL_F", 0) + NULLIF("SCH_ENR_BL_M", 0)) as bl_no_disab_sing_suspension_rate
    ,("SCH_DISCWODIS_REF_BL_F" + "SCH_DISCWODIS_REF_BL_M")  / (NULLIF("SCH_ENR_BL_F", 0) + NULLIF("SCH_ENR_BL_M", 0)) as bl_no_disab_le_referral_rate
    ,("SCH_DISCWODIS_REF_WH_F" + "SCH_DISCWODIS_REF_WH_M")  / (NULLIF("SCH_ENR_WH_F", 0) + NULLIF("SCH_ENR_WH_M", 0)) as wh_no_disab_le_referral_rate
